In [2]:
# Repo + LFS
!apt-get -qq update >/dev/null 2>&1
!apt-get -qq install -y git-lfs >/dev/null 2>&1
!git clone -q https://github.com/ronpolonsky/language-model-from-scratch.git >/dev/null 2>&1 || true
%cd language-model-from-scratch
!git lfs install >/dev/null 2>&1 && git lfs pull >/dev/null 2>&1

# Pin a compatible combo: NumPy 2.x + Torch 2.6.0 (CPU)
!pip -q uninstall -y torch torchvision torchaudio numpy >/dev/null 2>&1 || true
!pip -q install "numpy>=2.0.0,<3.0.0" --force-reinstall --no-cache-dir >/dev/null 2>&1
!pip -q install torch==2.6.0 --index-url https://download.pytorch.org/whl/cpu >/dev/null 2>&1


!sed -i '/^numpy[>=<~]/d; /^torch[>=<~]/d' interactive_app/requirements.txt >/dev/null 2>&1 || true
!pip -q install -r interactive_app/requirements.txt ipywidgets >/dev/null 2>&1

# Inline UI (ipywidgets)
import warnings
warnings.filterwarnings("ignore")  # hide non-critical Python warnings

import ipywidgets as w
from IPython.display import display, Markdown, clear_output
from my_code.inference_from_user_prompt import inference_from_input

# Title
display(Markdown("## Transformer Language Model Demo"))

# Inputs
prompt = w.Textarea(
    value="",
    placeholder="Type your prompt here…",
    layout=w.Layout(width="100%", height="140px"),
)

btn = w.Button(
    description="Generate",
    button_style="primary",
    layout=w.Layout(width="160px", height="40px"),
)

status = w.HTML("")        # small inline status
out = w.Output()

# “card” container (no external libs)
card = w.VBox(
    [
        prompt,
        w.HBox([btn, w.Box([status], layout=w.Layout(margin="0 0 0 12px"))]),
        out,
    ],
    layout=w.Layout(
        border="1px solid #e6e9ef",
        border_radius="12px",
        padding="16px",
        background_color="#fafcff",
        width="100%",
    ),
)

def on_click(_):
    with out:
        clear_output(wait=True)
        txt = prompt.value.strip()
        if not txt:
            status.value = "<span style='color:#b00020;'>Please enter a prompt.</span>"
            return
        status.value = "⏳ Generating…"
        try:
            result = inference_from_input(txt)
            status.value = ""
            display(Markdown("### Generated"))
            display(Markdown(f"```\n{result}\n```"))
        except Exception:
            status.value = ""
            display(Markdown(
                "**Error**: generation failed. Try a shorter prompt or rerun the cell."
            ))

btn.on_click(on_click)
display(card)


/content/language-model-from-scratch/language-model-from-scratch


## Transformer Language Model Demo